In [ ]:
import dfxml.fiwalk as fiwalk
import dfxml 
import sys
import pandas as pd
import numpy as np
import sqlite3
import hashlib
import pprint
from pprint import pprint
imagefile = open("/Volumes/Samsung_T5/M57/pat-2009-12-10.raw.raw")
data = ""
def process(fi): # fi=fileobject
    #offset = fi.contents().find(data)
    #if offset>0:
    print(fi.filename())
    
objs = fiwalk.fileobjects_using_sax(imagefile=imagefile)
print(len(objs))


dfxml_path = "/Volumes/Samsung_T5/M57/pat-2009-12-10.xml"
img_path = "/Volumes/Samsung_T5/argo/pat2.csv"

id = 27964
#Lets make sure all "files" are covered
# Lets store it a DB



files_cols = ['obj_id', 'partition','inode','filename','filesize']
block_hashes_cols = ['obj_id','img_offset','fs_offset','file_offset','len','md5','sha1']
file_df = pd.DataFrame(columns=files_cols)
block_hashes_df = pd.DataFrame(columns=block_hashes_cols)

sector_size = 512

counter = 0
print(len(objs))
#with open(img_path, 'rb') as img_h:
for obj in objs:
    if int(obj._tags['id']) == id:# and obj.filesize() == False:
        counter += 1
        #print(obj.filename())
        data = [obj._tags['id'], obj.partition(),obj.inode(), obj.filename(),  obj.filesize()]
        file_df.loc[len(file_df.index)] = data
        print(data)
        byterun = []
        persist_img_offset = 0
        persist_fs_offset = 0
        persist_file_offset = 0
        print(obj.byte_runs())
        for byterun1 in obj.byte_runs():
            pprint(vars(byterun1))


In [30]:
        print(len(obj.byte_runs()))

#for ebyterun in obj.byte_runs():
#    pprint(vars(ebyterun))


1


In [ ]:
id = 11580
#Lets make sure all "files" are covered
# Lets store it a DB
import pandas as pd
import numpy as np
import sqlite3
import hashlib
import pprint
import math
dfxml_path = "/Volumes/Samsung_T5/M57/pat-2009-12-10.xml"
img_csv = "/Volumes/Samsung_T5/argo/pat2.csv"
img_csv_df = pd.read_csv(img_csv)
img_path =  "/Volumes/Samsung_T5/M57/pat-2009-12-10.raw.raw"
files_cols = ['obj_id', 'partition','inode','filename','filesize']
block_hashes_cols = ['obj_id','img_offset','fs_offset','file_offset','len','md5','sha1']
file_df = pd.DataFrame(columns=files_cols)
block_hashes_df = pd.DataFrame(columns=block_hashes_cols)

sector_size = 512
from pprint import pprint
counter = 0
print(len(objs))
with open(img_path, 'rb') as img_h:
    for obj in objs:
        if int(obj._tags['id']) == id:# and obj.filesize() == False:
            counter += 1
            #print(obj.filename())
            data = [obj._tags['id'], obj.partition(),obj.inode(), obj.filename(),  obj.filesize()]
            file_df.loc[len(file_df.index)] = data
            print(data)
            byterun = []
            persist_img_offset = 0
            persist_fs_offset = 0
            persist_file_offset = 0
            remaining_len = obj.filesize()
            ebyterun_df = pd.DataFrame(columns=['obj_id','img_offset','fs_offset','file_offset','len','md5','sha1'])
            for ebyterun in obj.byte_runs():
                #pprint(vars(ebyterun)) 
                #byterun.append([ebyterun.img_offset,ebyterun.file_offset, ebyterun.len, ebyterun.fs_offset])
                byterun.append(ebyterun.file_offset)
                persist_file_offset = ebyterun.file_offset
                byterun.append(ebyterun.len)
                if hasattr(ebyterun,'img_offset'): 
                    byterun.append(ebyterun.img_offset)
                    if ebyterun.img_offset != None: persist_img_offset = ebyterun.img_offset
                if hasattr(ebyterun,'fs_offset'): 
                    byterun.append(ebyterun.fs_offset)
                    if ebyterun.fs_offset != None: persist_fs_offset = ebyterun.fs_offset
                if hasattr(ebyterun,'len'): 
                    byterun.append(ebyterun.len)
                    if ebyterun.len != None: persist_len = ebyterun.len
                #print("img_offset", img_offset, "file offset", ebyterun.file_offset)
                #print("file_offset", "len", "img_offset", "fs_offset")
                byterun_start = int(persist_img_offset / sector_size)
                byterun_end = int((math.ceil((persist_img_offset + ebyterun.len) / sector_size))) - 1
                len_run = np.arange(persist_len,0,-sector_size) #remaining_len
                sector_run = np.full(len(len_run), sector_size)
                len_run = np.minimum(len_run,sector_run)
                #len_run = min(sector_size,len_run.all())
                print("len_run",len(len_run), len(len_run)*sector_size)
                print("byterun_start", byterun_start,persist_img_offset, "byterun_end", byterun_end,  "len", ebyterun.len)
                print("ebyterun_df", ebyterun_df.shape, "img_csv", img_csv_df.loc[byterun_start:byterun_end,'img_offset'].shape )
                #print(img_csv_df.loc[byterun_start:byterun_end,:])
                print("img_csv_len",len(img_csv_df.loc[byterun_start:byterun_end, :]))
                ebyterun_df.loc[:,'img_sector_offset'] = np.arange(len(img_csv_df))
                ebyterun_df.loc[byterun_start:byterun_end,'obj_id'] = obj._tags['id']
                ebyterun_df.loc[byterun_start:byterun_end,'img_offset'] = img_csv_df.loc[byterun_start:byterun_end,'img_offset']
                ebyterun_df.loc[byterun_start:byterun_end,'fs_offset'] = np.arange(persist_fs_offset,persist_fs_offset+ebyterun.len,sector_size)
                ebyterun_df.loc[byterun_start:byterun_end,'file_offset'] = np.arange(persist_file_offset,persist_file_offset+ebyterun.len, sector_size)
                ebyterun_df.loc[byterun_start:byterun_end,'len'] = len_run
                #print("remaining_len",remaining_len)
                #remaining_len-=sector_size
                ebyterun_df.loc[byterun_start:byterun_end,'md5'] = img_csv_df.loc[byterun_start:byterun_end,'md5']
            print(ebyterun_df)    
'''
                remaining_len = ebyterun.len
                #cur_file_offset = 0
                #print("image offset", persist_img_offset)
                #print("file offset", ebyterun.file_offset)
                #print("len", ebyterun.len)
                #print()
                for pointer in range(persist_img_offset,persist_img_offset+ebyterun.len, sector_size):
                    #print(pointer)
                    persist_img_offset = pointer
                    
                    img_h.seek(pointer)
                    fsector = img_h.read(sector_size)
                    
                    hash_len = min(remaining_len,sector_size)
                    
                    sector_md5 = hashlib.md5(fsector).hexdigest()
                    sector_sha1 = hashlib.sha1(fsector).hexdigest()
                    data2 = [obj._tags['id'], pointer, persist_fs_offset, persist_file_offset, hash_len, sector_md5, sector_sha1]
                    persist_fs_offset+=sector_size 
                    persist_file_offset+=sector_size 
                    remaining_len -= sector_size
                    block_hashes_df.loc[len(block_hashes_df.index)] = data2

                print(counter, " out of ", len(objs), "id= ",obj._tags['id'])

    #print(len(file_df))
conn = sqlite3.connect("/Users/seunfuta/Downloads/M57/charlie-2009-12-11.db")
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS files (obj_id INTEGER, partition INTEGER, inode INTEGER, filename TEXT, filesize INTEGER)')
conn.commit()
cur.execute('CREATE TABLE IF NOT EXISTS block_hashes (obj_id INTEGER, img_offset INTEGER,fs_offset INTEGER, file_offset INTEGER,len INTEGER, md5 TEXT,sha1 TEXT)')
conn.commit()
file_df.to_sql('files', conn, if_exists='replace', index = False)
block_hashes_df.to_sql('block_hashes', conn, if_exists='replace', index = False)
conn.close()
'''

In [62]:
ebyterun_df.loc[13558039:13558046,:]


,obj_id,img_offset,fs_offset,file_offset,len,md5,sha1,img_sector_offset
13558039,1211,6941715968,6941683712,0,512,39dcab5c1d131f2a84db06959ae4a4d7,NaN,13558039
13558040,1211,6941716480,6941684224,512,512,111ef107791fd7ab654a04e91135a072,NaN,13558040
13558041,1211,6941716992,6941684736,1024,512,dbf0e745da11cd5810f2189cc2587534,NaN,13558041
13558042,1211,6941717504,6941685248,1536,512,274c3ccd287a94479a60414a1686cd42,NaN,13558042
13558043,1211,6941718016,6941685760,2048,512,227b5a49b52935d27e6f2ed538161d6e,NaN,13558043
13558044,1211,6941718528,6941686272,2560,512,7d67abbcd8d5c7b03d6006bf00f7562a,NaN,13558044
13558045,1211,6941719040,6941686784,3072,512,90af5b3c60b4ccd524ade98ecb5f0691,NaN,13558045
13558046,1211,6941719552,6941687296,3584,121,fb6f3f4d96419ea9b887ca9defa28911,NaN,13558046


In [55]:
ebyterun_df.loc[13558039:13558046,:]

,obj_id,img_offset,fs_offset,file_offset,len,md5,sha1,img_sector_offset
13558039,1211,6941715968,6941683712,0,True,39dcab5c1d131f2a84db06959ae4a4d7,NaN,13558039
13558040,1211,6941716480,6941684224,512,True,111ef107791fd7ab654a04e91135a072,NaN,13558040
13558041,1211,6941716992,6941684736,1024,True,dbf0e745da11cd5810f2189cc2587534,NaN,13558041
13558042,1211,6941717504,6941685248,1536,True,274c3ccd287a94479a60414a1686cd42,NaN,13558042
13558043,1211,6941718016,6941685760,2048,True,227b5a49b52935d27e6f2ed538161d6e,NaN,13558043
13558044,1211,6941718528,6941686272,2560,True,7d67abbcd8d5c7b03d6006bf00f7562a,NaN,13558044
13558045,1211,6941719040,6941686784,3072,True,90af5b3c60b4ccd524ade98ecb5f0691,NaN,13558045
13558046,1211,6941719552,6941687296,3584,True,fb6f3f4d96419ea9b887ca9defa28911,NaN,13558046


In [ ]:
        '''
        for ebyterun in obj.byte_runs():
            pprint(vars(ebyterun)) 
        
            byterun.append(ebyterun.file_offset)
            persist_file_offset = ebyterun.file_offset
            byterun.append(ebyterun.len)
            if hasattr(ebyterun,'img_offset'): 
                byterun.append(ebyterun.img_offset)
                if ebyterun.img_offset != None: persist_img_offset = ebyterun.img_offset
            if hasattr(ebyterun,'fs_offset'): 
                byterun.append(ebyterun.fs_offset)
                if ebyterun.fs_offset != None: persist_fs_offset = ebyterun.fs_offset

            
            remaining_len = ebyterun.len

            for pointer in range(persist_img_offset,persist_img_offset+ebyterun.len, sector_size):
                #print(pointer)
                persist_img_offset = pointer
                
                img_h.seek(pointer)
                fsector = img_h.read(sector_size)
                
                hash_len = min(remaining_len,sector_size)
                
                sector_md5 = hashlib.md5(fsector).hexdigest()
                sector_sha1 = hashlib.sha1(fsector).hexdigest()
                data2 = [obj._tags['id'], pointer, persist_fs_offset, persist_file_offset, hash_len, sector_md5, sector_sha1]
                persist_fs_offset+=sector_size 
                persist_file_offset+=sector_size 
                remaining_len -= sector_size
                block_hashes_df.loc[len(block_hashes_df.index)] = data2
            
            print(counter, " out of ", len(objs), "id= ",obj._tags['id'])
        '''

In [ ]:
        '''
#print(len(file_df))
'''
conn = sqlite3.connect("/Volumes/Samsung_T5/M57/pat-2009-12-10.db")
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS files (obj_id INTEGER, partition INTEGER, inode INTEGER, filename TEXT, filesize INTEGER)')
conn.commit()
cur.execute('CREATE TABLE IF NOT EXISTS block_hashes (obj_id INTEGER, img_offset INTEGER,fs_offset INTEGER, file_offset INTEGER,len INTEGER, md5 TEXT,sha1 TEXT)')
conn.commit()
file_df.to_sql('files', conn, if_exists='replace', index = False)
block_hashes_df.to_sql('block_hashes', conn, if_exists='replace', index = False)
conn.close()
'''

In [ ]:
def process_fi(self,fi):
    global options
    # Filter out specific filenames create by TSK that are not of use
    if ignore_filename(fi.filename(), self.include_dotdirs):
        return 

    dprint("processing %s" % str(fi))
    
    # See if the filename changed its hash code
    changed = False
    if not fi.allocated():
        return # only look at allocated files

    # Remember the file for the next generation
    self.new_fnames[fi.filename()] = fi
    self.new_inodes[(fi.partition(), fi.inode())] = fi
    self.new_fi_tally += 1

    # See if a file with this filename had its contents change or properties changed
    ofi = self.fnames.get(fi.filename(),None)
    if ofi:
        dprint("   found ofi")
        any_diff = False
        if ofi.sha1()!=fi.sha1():
            dprint("      >>> sha1 changed")
            self.changed_content.add((ofi,fi))
            any_diff = True
        elif ofi.atime() != fi.atime() or \
                ofi.mtime() != fi.mtime() or \
                ofi.crtime() != fi.crtime() or \
                ofi.ctime() != fi.ctime():
            dprint("      >>> time changed")
            self.changed_properties.add((ofi,fi))
            any_diff = True

        if any_diff:
            #Count the types of changes that happened
            if ofi.filesize() != fi.filesize():
                self.changed_filesize_tally += 1
            if ofi.sha1() != fi.sha1():
                if ofi.is_dir():
                    self.changed_dir_sha1_tally += 1
                elif ofi.is_file():
                    self.changed_file_sha1_tally += 1
            if ofi.mtime() != fi.mtime():
                self.changed_mtime_tally += 1
            if ofi.atime() != fi.atime():
                self.changed_atime_tally += 1
            if ofi.ctime() != fi.ctime():
                self.changed_ctime_tally += 1
            if ofi.crtime() != fi.crtime():
                self.changed_crtime_tally += 1
            if ofi.byte_runs() and fi.byte_runs():
                brdiff = 0
                ofirstbr = ofi.byte_runs()[0]
                nfirstbr =  fi.byte_runs()[0]
                try:
                    if ofirstbr.file_offset == nfirstbr.file_offset:
                        brdiff = 1
                    if ofirstbr.img_offset == nfirstbr.img_offset:
                        brdiff = 1
                    if ofirstbr.fs_offset == nfirstbr.fs_offset:
                        brdiff = 1
                except:
                    pass
                self.changed_first_byterun_tally += brdiff
        

    # If a new file, note that (and optionally add to the timeline)
    if not ofi:
        self.new_files.add(fi)
        if self.timeline:
            create_time = fi.crtime()
            if not create_time: create_time = fi.ctime()
            if not create_time: create_time = fi.mtime()
            self.timeline.add((create_time,fi.filename(),"created"))

    # Delete files we have seen (so we can find out the files that were deleted)
    if fi.filename() in self.fnames:
        del self.fnames[fi.filename()]

    # Look for files that were renamed
    ofi = self.inodes.get((fi.partition(), fi.inode()),None)
    if ofi and ofi.filename() != fi.filename() and ofi.sha1()==fi.sha1():
        #Never consider current-directory or parent-directory for rename operations.  Because we match on partition+inode numbers, these trivially match.
        if not (fi.filename().endswith("/.") or fi.filename().endswith("/..") or ofi.filename().endswith("/.") or ofi.filename().endswith("/..")):
            self.renamed_files.add((ofi,fi))

def process(self,fname):
    self.prior_fname = self.current_fname
    self.current_fname = fname
    if fname.endswith("xml"):
        with open(fname,'rb') as xmlfile:
            for fi in dfxml.iter_dfxml(xmlfile, preserve_elements=True):
                self.process_fi(fi)
    else:
        fiwalk.fiwalk_using_sax(imagefile=open(fname,'rb'), flags=fiwalk.ALLOC_ONLY, callback=self.process_fi)

In [ ]:

#Lets make sure all "files" are covered
# Lets store it a DB
import pandas as pd
import numpy as np
import sqlite3
import hashlib
import pprint
import math
from pprint import pprint
import dfxml 


id = 11580
dfxml_path = "/Volumes/Samsung_T5/M57/pat-2009-12-10.raw.xml"
img_csv = "/Volumes/Samsung_T5/argo/pat2.csv"
img_csv_df = pd.read_csv(img_csv)
img_path =  "/Volumes/Samsung_T5/M57/pat-2009-12-10.raw.raw"
files_cols = ['obj_id', 'partition','inode','filename','filesize']
block_hashes_cols = ['obj_id','img_offset','fs_offset','file_offset','len','md5','sha1']
file_df = pd.DataFrame(columns=files_cols)
block_hashes_df = pd.DataFrame(columns=block_hashes_cols)

sector_size = 512
counter = 0
#print(len(objs))

def process_fi(obj):
    global options
    # Filter out specific filenames create by TSK that are not of use
    if not obj.allocated():
        return
    print("processing %s" % str(obj))
    
    # See if the filename changed its hash code
    changed = False
    if not obj.allocated():
        return # only look at allocated files

    if int(obj._tags['id']) == id:# and obj.filesize() == False:
        #counter += 1
        #print(obj.filename())
        data = [obj._tags['id'], obj.partition(),obj.inode(), obj.filename(),  obj.filesize()]
        file_df.loc[len(file_df.index)] = data
        print(data)
        byterun = []
        persist_img_offset = 0
        persist_fs_offset = 0
        persist_file_offset = 0
        remaining_len = obj.filesize()
        ebyterun_df = pd.DataFrame(columns=['obj_id','img_offset','fs_offset','file_offset','len','md5','sha1'])
        for ebyterun in obj.byte_runs():
            #pprint(vars(ebyterun)) 
            #byterun.append([ebyterun.img_offset,ebyterun.file_offset, ebyterun.len, ebyterun.fs_offset])
            byterun.append(ebyterun.file_offset)
            persist_file_offset = ebyterun.file_offset
            byterun.append(ebyterun.len)
            if hasattr(ebyterun,'img_offset'): 
                byterun.append(ebyterun.img_offset)
                if ebyterun.img_offset != None: persist_img_offset = ebyterun.img_offset
            if hasattr(ebyterun,'fs_offset'): 
                byterun.append(ebyterun.fs_offset)
                if ebyterun.fs_offset != None: persist_fs_offset = ebyterun.fs_offset
            if hasattr(ebyterun,'len'): 
                byterun.append(ebyterun.len)
                if ebyterun.len != None: persist_len = ebyterun.len
            #print("img_offset", img_offset, "file offset", ebyterun.file_offset)
            #print("file_offset", "len", "img_offset", "fs_offset")
            byterun_start = int(persist_img_offset / sector_size)
            byterun_end = int((math.ceil((persist_img_offset + ebyterun.len) / sector_size))) - 1
            len_run = np.arange(persist_len,0,-sector_size) #remaining_len
            sector_run = np.full(len(len_run), sector_size)
            len_run = np.minimum(len_run,sector_run)
            #len_run = min(sector_size,len_run.all())
            print("len_run",len(len_run), len(len_run)*sector_size)
            print("byterun_start", byterun_start,persist_img_offset, "byterun_end", byterun_end,  "len", ebyterun.len)
            print("ebyterun_df", ebyterun_df.shape, "img_csv", img_csv_df.loc[byterun_start:byterun_end,'img_offset'].shape )
            #print(img_csv_df.loc[byterun_start:byterun_end,:])
            print("img_csv_len",len(img_csv_df.loc[byterun_start:byterun_end, :]))
            ebyterun_df.loc[:,'img_sector_offset'] = np.arange(len(img_csv_df))
            ebyterun_df.loc[byterun_start:byterun_end,'obj_id'] = obj._tags['id']
            ebyterun_df.loc[byterun_start:byterun_end,'img_offset'] = img_csv_df.loc[byterun_start:byterun_end,'img_offset']
            ebyterun_df.loc[byterun_start:byterun_end,'fs_offset'] = np.arange(persist_fs_offset,persist_fs_offset+ebyterun.len,sector_size)
            ebyterun_df.loc[byterun_start:byterun_end,'file_offset'] = np.arange(persist_file_offset,persist_file_offset+ebyterun.len, sector_size)
            ebyterun_df.loc[byterun_start:byterun_end,'len'] = len_run
            #print("remaining_len",remaining_len)
            #remaining_len-=sector_size
            ebyterun_df.loc[byterun_start:byterun_end,'md5'] = img_csv_df.loc[byterun_start:byterun_end,'md5']
        print(ebyterun_df)    


def process(fname):
    if fname.endswith("xml"):
        with open(fname,'rb') as xmlfile:
            for fi in dfxml.iter_dfxml(xmlfile, preserve_elements=True):
                process_fi(fi)
    else:
        fiwalk.fiwalk_using_sax(imagefile=open(fname,'rb'), flags=fiwalk.ALLOC_ONLY, callback=self.process_fi)
#objs = fiwalk.fileobjects_using_sax(imagefile=imagefile)

#with open(img_path, 'rb') as img_h:
process(dfxml_path)

    